In [ ]:
from azure.storage.filedatalake import DataLakeServiceClient 
from azure.core.exceptions import ResourceNotFoundError 
from datetime import datetime, timedelta
import time

  

def file_arrival_check(account_name, account_key, container_name, source_path): 

    # Initialize DataLake Service Client 

    service_client = DataLakeServiceClient( 

        account_url=f"https://{account_name}.dfs.core.windows.net", 

        credential=account_key 

    )

    # Get filesystem client 

    file_system_client = service_client.get_file_system_client(container_name)
    max_wait_time = timedelta(minutes=1)
    check_interval = timedelta(seconds=10)
    start_time = datetime.now()
    found_source_path = False 


  # Step 1: Check if source_path exists 

    while datetime.now() - start_time < max_wait_time:
        try:
            source_dir_client = file_system_client.get_directory_client(source_path)
            source_dir_client.get_directory_properties()
            found_source_path = True
            print(f"Source path '{source_path}' exists in container '{container_name}'.")
            break
        except ResourceNotFoundError:
            time.sleep(check_interval.total_seconds())
        except Exception:
            time.sleep(check_interval.total_seconds())
    if not found_source_path:
        print(f"Source path '{source_path}' does not exist in container '{container_name}'.")
        return False
    return True

if __name__ == "__main__":
    # Azure Storage Account Details
    ACCOUNT_NAME = "stgipplgt10153"
    ACCOUNT_KEY = "sp=rcwdl&st=2024-08-04T11:13:44Z&se=2025-08-05T19:13:44Z&spr=https&sv=2022-11-02&sr=c&sig=LkP5vz9SuubId3JNQCfh1GfOxRO9pFiXac0JecySCMw%3D"
    CONTAINER_NAME = "stgippl-azrgt10153"
    SOURCE_PATH = "dev/inbound/stolinedetails/stolinedetails_202505111906/"
    result = file_arrival_check(ACCOUNT_NAME, ACCOUNT_KEY, CONTAINER_NAME, SOURCE_PATH)
    if result:
        print("Proceeding with file processing...")
    else:
        print("File processing aborted due to source path not found.")
